In [1]:
import bayesflow as bf
import tensorflow_probability as tfp
import tensorflow as tf
import numpy as np
import torch
from torch.distributions import Normal, Independent,InverseGamma

import rpy2.robjects as robjects
import numpy as np

/opt/anaconda3/envs/bayesflow/lib/python3.11/site-packages/bayesflow/trainers.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [1]:
import rpy2.robjects as robjects
import numpy as np

# 从 R 中加载数据
robjects.r('load("SpainPROV_CerebrovascularDiseases.Rdata")')

Rs = robjects.r('as.matrix(Rs)')

# 将 Rs 转换为 NumPy 数组
Rs_numpy = np.array(Rs)

# 检查矩阵的维度
print("Rs_numpy shape:", Rs_numpy.shape)


R[write to console]: Loading required package: Matrix



Rs_numpy shape: (47, 47)


In [2]:
# 计算特征分解
eigenvalues, eigenvectors = np.linalg.eig(Rs_numpy)

# 打印特征值和特征向量
print("Eigenvalues:", eigenvalues)
print("Eigenvectors:", eigenvectors)

# 计算协方差矩阵 Q
Q = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T  # Q = V * D * V.T
print("Covariance matrix Q:", Q)


Eigenvalues: [4.02898372e+00 3.91760840e+00 1.16270678e-16 8.70103967e-02
 1.36471249e-01 3.68894189e+00 2.84373915e-01 3.35903480e-01
 4.24394939e-01 3.57526766e+00 3.48700471e+00 5.49480100e-01
 6.86479773e-01 3.28806127e+00 3.24627671e+00 3.22501234e+00
 8.25429149e-01 9.23508897e-01 9.38544804e-01 1.00121440e+00
 1.08337936e+00 3.09257535e+00 1.18549682e+00 1.31239606e+00
 1.37151192e+00 1.41839871e+00 1.47548171e+00 2.94744613e+00
 2.90831092e+00 2.81326532e+00 2.78203196e+00 2.70367067e+00
 2.65184367e+00 2.55747008e+00 2.53141064e+00 2.45001418e+00
 2.38822842e+00 2.32899568e+00 2.26667887e+00 2.10897083e+00
 2.09243660e+00 1.96124040e+00 1.90970136e+00 1.87819751e+00
 1.75053418e+00 1.68927737e+00 1.71788199e+00]
Eigenvectors: [[ 0.09193583  0.12350732 -0.14586499 ...  0.16560432  0.01980325
   0.00368928]
 [ 0.05232506 -0.14409124 -0.14586499 ...  0.07575858 -0.08023009
   0.07457773]
 [-0.00563195  0.02052873 -0.14586499 ... -0.04477425  0.08674872
  -0.07251008]
 ...
 [ 0.07

In [3]:
eigenvalues, _ = np.linalg.eig(Q)
print("Eigenvalues of Q:", eigenvalues)


Eigenvalues of Q: [ 4.02898372e+00  3.91760840e+00 -1.27713007e-17  8.70103967e-02
  1.36471249e-01  3.68894189e+00  2.84373915e-01  3.35903480e-01
  4.24394939e-01  3.57526766e+00  3.48700471e+00  5.49480100e-01
  6.86479773e-01  3.28806127e+00  3.24627671e+00  3.22501234e+00
  8.25429149e-01  9.23508897e-01  9.38544804e-01  1.00121440e+00
  1.08337936e+00  3.09257535e+00  1.18549682e+00  1.31239606e+00
  1.37151192e+00  1.41839871e+00  1.47548171e+00  2.94744613e+00
  2.90831092e+00  2.81326532e+00  2.78203196e+00  2.70367067e+00
  2.65184367e+00  2.55747008e+00  2.53141064e+00  2.45001418e+00
  2.38822842e+00  2.32899568e+00  2.26667887e+00  2.10897083e+00
  2.09243660e+00  1.96124040e+00  1.90970136e+00  1.87819751e+00
  1.75053418e+00  1.68927737e+00  1.71788199e+00]


In [7]:
# # 如果有负特征值，添加一个非常小的正数到对角线
Q_corrected = Q + np.eye(Q.shape[0]) * 1e-6  # 1e-6 是一个小的常数
# 
# 重新计算特征值
eigenvalues, _ = np.linalg.eig(Q_corrected)
print("Corrected Eigenvalues of Q:", eigenvalues)


Corrected Eigenvalues of Q: [4.02898472e+00 3.91760940e+00 1.00000000e-06 8.70113967e-02
 1.36472249e-01 3.68894289e+00 2.84374915e-01 3.35904480e-01
 4.24395939e-01 3.57526866e+00 3.48700571e+00 5.49481100e-01
 6.86480773e-01 3.28806227e+00 3.24627771e+00 3.22501334e+00
 8.25430149e-01 9.23509897e-01 9.38545804e-01 1.00121540e+00
 1.08338036e+00 3.09257635e+00 1.18549782e+00 1.31239706e+00
 1.37151292e+00 1.41839971e+00 1.47548271e+00 2.94744713e+00
 2.90831192e+00 2.81326632e+00 2.78203296e+00 2.70367167e+00
 2.65184467e+00 2.55747108e+00 2.53141164e+00 2.45001518e+00
 2.38822942e+00 2.32899668e+00 2.26667987e+00 2.10897183e+00
 2.09243760e+00 1.96124140e+00 1.90970236e+00 1.87819851e+00
 1.75053518e+00 1.68927837e+00 1.71788299e+00]


In [5]:
import tensorflow_probability as tfp
import tensorflow as tf

# 生成随机效应样本的函数，模拟 rMVNormP_eigen
def rMVNormP_eigen(s, mu, Q):
    mvn = tfp.distributions.MultivariateNormalFullCovariance(
        loc=mu, covariance_matrix=Q
    )
    samples = mvn.sample(s)
    return samples

# 生成随机效应样本
s = 100  # 生成 100 个样本
mu = np.zeros(Rs_numpy.shape[0])  # 均值为零

samples = rMVNormP_eigen(s=s, mu=mu, Q=Q)

# 计算 log.risk
intercept = 0
log_risk = intercept + samples
print("log.risk:", log_risk)


Instructions for updating:
`MultivariateNormalFullCovariance` is deprecated, use `MultivariateNormalTriL(loc=loc, scale_tril=tf.linalg.cholesky(covariance_matrix))` instead.
log.risk: tf.Tensor(
[[ 0.27719738 -0.27776796 -0.23444625 ... -1.84332433  2.70626042
  -2.19835147]
 [-2.68584589  0.06403672 -0.24348869 ...  3.09788747 -2.37001795
   1.17701691]
 [-1.16399424  0.55263001  0.71192311 ... -0.01662258  0.35230198
   1.20599764]
 ...
 [ 0.82818925  1.54549938  0.66996058 ... -1.01692731 -0.19067121
   2.36206349]
 [-1.47382973  0.80086691 -1.16054818 ... -0.35046093 -2.91210992
   0.94168222]
 [-2.21587004  0.28695264 -1.61796679 ... -0.32736237 -1.40173897
  -0.04374804]], shape=(100, 47), dtype=float64)


In [6]:
mu = np.zeros(Rs_numpy.shape[0], dtype=np.float32) 

In [ ]:
# import tensorflow as tf
# import tensorflow_probability as tfp
# import bayesflow as bf
# import numpy as np
# 
# # ----------------------
# # 定义先验分布 (Prior)
# # ----------------------
# # def prior_fun(batch_size=1):
# #     """
# #     定义 intercept 的先验分布:
# #     - intercept (\eta): Uniform(low, high)
# #     """
# #     low, high = -2.0, 2.0  # 设定 intercept 的范围
# #     intercept_dist = tfp.distributions.Uniform(low=low, high=high)
# #     intercept_samples = intercept_dist.sample(batch_size)
# #     
# #     # 返回形状为 (batch_size, 1) 的先验样本
# #     return tf.expand_dims(intercept_samples, axis=-1)
# 
# def prior_fun(batch_size=47):
#     """
#     定义 intercept 的先验分布:
#     - intercept (\eta): Uniform(low, high)
#     """
#     low, high = 1.0, 2.0  # 设定 intercept 的范围
#     tau_dist = tfp.distributions.Uniform(low=low, high=high)
#     tau_samples = tau_dist.sample(1)
#     
#     # Q_tau_adjusted = Q * tau_samples
#     
#     mvn = tfp.distributions.MultivariateNormalFullCovariance(
#         loc=mu, covariance_matrix=Q*tau_samples
#     )
#  
#     samples = mvn.sample()
#     intercept = 0
#     log_risk = intercept + samples
#     
#     tau_samples_expanded = tf.broadcast_to(tau_samples, (1, Rs_numpy.shape[0])) # 广播 tau_samples
#     
#     # 返回 log.risk 和 tau_samples，保证它们的形状一致
#     return tf.stack([log_risk, tau_samples_expanded], axis=-1)
# 
# 
# # ----------------------
# # 定义模拟器 (Simulator)
# # ----------------------
# def likelihood_fun(params, n_obs=100):
#     """
#     模拟器 (生成观测数据):
#     - 根据 intercept 和固定 tau 值生成泊松分布的观测数据
#     """
#     log_risk = params[:,:,0]  # 提取log_risk
# 
#     # 生成观测病例数: O_i | r_i ~ Poisson(e^log_risk)
#     observed_cases = tf.random.poisson(
#         lam=tf.exp(log_risk), shape=[n_obs]
#     )  # 形状: (n_obs, batch_size, n_areas)
# 
#     # 返回形状为 (batch_size, n_obs, n_areas)
#     return tf.transpose(observed_cases, perm=[1, 0, 2])
# 
# # ----------------------
# # 定义生成模型 (Generative Model)
# # ----------------------
# prior = bf.simulation.Prior(prior_fun=prior_fun)
# simulator = bf.simulation.Simulator(simulator_fun=likelihood_fun)
# model = bf.simulation.GenerativeModel(prior=prior, simulator=simulator)
# 
# # ----------------------
# # 定义总结网络 (Summary Network)
# # ----------------------
# summary_net = bf.networks.DeepSet(summary_dim=10)
# 
# # ----------------------
# # 定义推断网络 (Inference Network)
# # ----------------------
# inference_net = bf.networks.InvertibleNetwork(
#     num_params=1,          # 只推断 1 个参数 intercept
#     num_coupling_layers=4  # 网络的耦合层数量
# )
# 
# # ----------------------
# # 定义后验推断器 (Posterior Amortizer)
# # ----------------------
# amortizer = bf.amortizers.AmortizedPosterior(inference_net, summary_net)
# 
# # ----------------------
# # 定义 Trainer (训练器)
# # ----------------------
# trainer = bf.trainers.Trainer(amortizer=amortizer, generative_model=model)
# 
# # 进行一次前向推理，验证生成模型是否工作
# _ = model(batch_size=47)  # 验证生成模型是否正常工作
# 
# # ---------------------- 
# # 不进行训练，只做一次数据生成
# # ----------------------
# # 调用模型，生成一个批次的数据
# generated_data = model(batch_size=47)
# 
# # 打印生成的数据
# print("Generated data shape:", generated_data.shape)
# 


INFO:root:Performing 2 pilot runs with the anonymous model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 3, 47, 2)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 3, 100, 47)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional simulation batchable context provided.
INFO:root:Performing a consistency check with provided components...
2025-02-03 15:40:55.656633: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.


In [ ]:
# import tensorflow as tf
# import tensorflow_probability as tfp
# import bayesflow as bf
# 
# def prior_fun(batch_size=1):
#     """
#     返回形状 [batch_size, 47] 的 log_risk 样本
#     """
#     # 确保 Q 是一个正定矩阵
#     mvn = tfp.distributions.MultivariateNormalFullCovariance(
#         loc=tf.constant(mu, dtype=tf.float32),  # [47]
#         covariance_matrix=tf.constant(Q, dtype=tf.float32)  # [47,47], 正定
#     )
#     # sample => [batch_size, 47]
#     log_risk_samples = mvn.sample(batch_size)
#     return log_risk_samples
# 
# def likelihood_fun(params, n_obs=100):
#     """
#     params: 形状 [batch_size, 47], 表示每个batch的一组 log_risk
#     返回:   形状 [batch_size, n_obs, 47]
#     """
#     # tf.exp(log_risk) => [batch_size, 47]
#     lam = tf.exp(params)
# 
#     # 调用 tf.random.poisson
#     # shape=[n_obs] 意味着对每个 batch 的 47个参数，生成 n_obs 次观测
#     # 返回张量形状默认会是 [n_obs, batch_size, 47] 或者 [batch_size, n_obs, 47]，依赖TF版本及broadcast。
#     observed_cases = tf.random.poisson(lam=lam, shape=[n_obs])  
#     # 如果结果是 [n_obs, batch_size, 47]，则转置到 [batch_size, n_obs, 47]
#     observed_cases = tf.transpose(observed_cases, perm=[1, 0, 2])
# 
#     return observed_cases
# 
# # 1) 将 prior_fun 包装成 BayesFlow 的 Prior
# prior = bf.simulation.Prior(prior_fun=prior_fun)
# 
# # 2) 将 likelihood_fun 包装成 BayesFlow 的 Simulator
# simulator = bf.simulation.Simulator(simulator_fun=likelihood_fun)
# 
# # 3) 组装成 generative model
# model = bf.simulation.GenerativeModel(prior=prior, simulator=simulator)
# 
# summary_net = bf.networks.DeepSet(
#     summary_dim=10
#     # 其余超参可改，如 num_layers, hidden_dim 等
# )
# 
# inference_net = bf.networks.InvertibleNetwork(
#     num_params=47,            # 重点！要匹配你先验的维度
#     num_coupling_layers=4     # 你可酌情调节
#     # 还可以指定 hidden_layers, etc.
# )
# 
# amortizer = bf.amortizers.AmortizedPosterior(inference_net, summary_net)
# trainer = bf.trainers.Trainer(amortizer=amortizer, generative_model=model)
# 
# # 做一次测试生成
# test_data = model(batch_size=8)  
# # shape = [8, 100, 47]
# print("test_data shape:", test_data.shape)
# 
# # 现在进行 online training
# # 注意: 这里只是演示, 你可根据需要调节 n_steps, batch_size 等
# trainer.train_online(n_steps=100, batch_size=8)
# 


In [19]:
import tensorflow as tf
import tensorflow_probability as tfp
import bayesflow as bf

def prior_fun(batch_size=1):
    """
    返回形状 [batch_size, 47] 的 log_risk 样本
    """
    # 确保 Q 是一个正定矩阵
    mvn = tfp.distributions.MultivariateNormalFullCovariance(
        loc=tf.constant(mu, dtype=tf.float32),  # [47]
        covariance_matrix=tf.constant(Q, dtype=tf.float32)  # [47,47], 正定
    )
    # sample => [batch_size, 47]
    log_risk_samples = mvn.sample(batch_size)
    return log_risk_samples

def likelihood_fun(params, n_obs=100):
    """
    params: 形状 [batch_size, 47], 表示每个batch的一组 log_risk
    返回:   形状 [batch_size, n_obs, 47]
    """
    # tf.exp(log_risk) => [batch_size, 47]
    lam = tf.exp(params)

    # 调用 tf.random.poisson
    # shape=[n_obs] 意味着对每个 batch 的 47个参数，生成 n_obs 次观测
    # 返回张量形状默认会是 [n_obs, batch_size, 47] 或者 [batch_size, n_obs, 47]，依赖TF版本及broadcast。
    observed_cases = tf.random.poisson(lam=lam, shape=[n_obs])  
    # 如果结果是 [n_obs, batch_size, 47]，则转置到 [batch_size, n_obs, 47]
    observed_cases = tf.transpose(observed_cases, perm=[1, 0, 2])

    return observed_cases

def simple_configurator(model_output):
    # 取出先验采样
    prior_draws = model_output["prior_draws"]  # [b, 1, 47]
    prior_draws = tf.squeeze(prior_draws, axis=1)  # => [b, 47]
    model_output["parameters"] = prior_draws       # 使BayesFlow识别到参数

    # 取出模拟数据
    sim_data = model_output["sim_data"]  # [b, 1, n_obs, 47]
    sim_data = tf.squeeze(sim_data, axis=1)  # => [b, n_obs, 47]

    # 告诉 BayesFlow：把这份数据当作 summary_conditions
    model_output["summary_conditions"] = sim_data
    # 或者你想保留 sim_data 这个键也行：
    # model_output["sim_data"] = sim_data

    return model_output




# 1) 将 prior_fun 包装成 BayesFlow 的 Prior
prior = bf.simulation.Prior(prior_fun=prior_fun)

# 2) 将 likelihood_fun 包装成 BayesFlow 的 Simulator
simulator = bf.simulation.Simulator(simulator_fun=likelihood_fun)

# 3) 组装成 generative model
model = bf.simulation.GenerativeModel(prior=prior, simulator=simulator)

summary_net = bf.networks.SetTransformer(
    input_dim= 47
)

inference_net = bf.networks.InvertibleNetwork(
    num_params=47,            # 重点！要匹配你先验的维度
    num_coupling_layers=4     # 你可酌情调节
    # 还可以指定 hidden_layers, etc.
)

amortizer = bf.amortizers.AmortizedPosterior(inference_net, summary_net)
trainer = bf.trainers.Trainer(
    amortizer=amortizer, 
    generative_model=model,
    configurator=simple_configurator  # <-- 指定自定义的configurator
)


# 做一次测试生成
test_data = model(batch_size=8)  
# shape = [8, 100, 47]
print("test_data:", test_data)


INFO:root:Performing 2 pilot runs with the anonymous model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 1, 47)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 1, 100, 47)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional simulation batchable context provided.
INFO:root:Performing a consistency check with provided components...
INFO:root:Done.


test_data: {'prior_non_batchable_context': None, 'prior_batchable_context': None, 'prior_draws': array([[[ 0.8935258 , -1.6783502 , -0.02551815, -1.4702312 ,
         -0.1919902 , -2.8637094 , -1.406093  , -2.9898899 ,
         -0.13486409, -0.31954703, -0.35830936, -0.17603141,
          1.1533508 , -2.029516  , -0.7967203 , -0.5755219 ,
          0.8437338 , -0.30460304, -0.60200655,  1.7435305 ,
          0.13680017, -0.3794902 ,  0.29123542,  0.8843081 ,
          0.20354742,  0.9841508 ,  1.5680236 ,  2.1809146 ,
          1.537014  , -0.4646935 ,  2.1282642 ,  0.4366497 ,
         -1.5883278 , -0.05516565, -1.5440953 , -0.35173124,
         -1.1491446 , -1.1372361 , -0.6114719 ,  1.752664  ,
         -0.8746476 ,  0.62201416,  1.4713683 ,  3.758645  ,
          1.5928798 , -0.6585144 ,  0.5551379 ]],

       [[ 0.19516928, -1.6067    , -2.0314894 , -1.4951557 ,
         -2.6858666 ,  0.8495594 , -0.60075414,  0.926971  ,
          1.1180106 ,  1.0197097 ,  0.04421227,  0.3807655 

In [22]:
# ----------------------
# 开始训练
# ----------------------
history = trainer.train_online(
    epochs=10,                # 总训练轮数
    iterations_per_epoch=500,  # 每轮训练的迭代次数
    batch_size=8,            # 每次训练的样本批量大小
    validation_sims=200       # 验证集样本数量
)

print("训练完成！")


INFO:root:Generated 200 simulations for validation.
Training epoch 1: 100%|██████████| 500/500 [00:38<00:00, 13.09it/s, Epoch: 1, Iter: 500,Loss: 72.465,W.Decay: 0.286,Avg.Loss: 74.362,Avg.W.Decay: 0.286,LR: 4.66E-04]
INFO:root:Validation, Epoch: 1, Loss: 73.333
Training epoch 2: 100%|██████████| 500/500 [00:36<00:00, 13.88it/s, Epoch: 2, Iter: 500,Loss: 73.939,W.Decay: 0.284,Avg.Loss: 72.710,Avg.W.Decay: 0.285,LR: 4.17E-04]
INFO:root:Validation, Epoch: 2, Loss: 72.113
Training epoch 3: 100%|██████████| 500/500 [00:35<00:00, 13.91it/s, Epoch: 3, Iter: 500,Loss: 75.398,W.Decay: 0.284,Avg.Loss: 71.628,Avg.W.Decay: 0.284,LR: 3.51E-04]
INFO:root:Validation, Epoch: 3, Loss: 71.179
Training epoch 4: 100%|██████████| 500/500 [00:36<00:00, 13.58it/s, Epoch: 4, Iter: 500,Loss: 73.615,W.Decay: 0.283,Avg.Loss: 70.655,Avg.W.Decay: 0.283,LR: 2.75E-04]
INFO:root:Validation, Epoch: 4, Loss: 69.959
Training epoch 5: 100%|██████████| 500/500 [00:36<00:00, 13.70it/s, Epoch: 5, Iter: 500,Loss: 70.965,W.D

训练完成！


/opt/anaconda3/envs/bayesflow/lib/python3.11/site-packages/bayesflow/helper_classes.py:683: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  losses_list = pd.concat(losses_list, axis=0).value.to_list()


In [23]:
# 1) 生成一个测试集
test_size = 2000
test_batch = model(batch_size=test_size)
# test_batch 里包含: 
#   test_batch["prior_draws"]       # shape [test_size, 1, 47] (若 n_draws=1)
#   test_batch["sim_data"]          # shape [test_size, 1, n_obs, 47]
#   可能还有 "meta_info" 等别的字段

# 2) 用 configurator 重整形状
test_batch = simple_configurator(test_batch)
# 现在:
#   test_batch["parameters"]        # shape [test_size, 47]
#   test_batch["summary_conditions"]# shape [test_size, n_obs, 47]

# 3) 分离出真参数和观测
theta_true = test_batch["parameters"]         # [test_size, 47]
x_test     = test_batch["summary_conditions"] # [test_size, n_obs, 47]


In [26]:
input_dict = {"summary_conditions": x_test}
posterior_samples = amortizer.sample(input_dict, n_samples=100, to_numpy=True)

# theta_samples 形状: [test_size, n_posterior_samples, 47]


In [28]:
import numpy as np
# 后验采样:
input_dict = {"summary_conditions": x_test}  # 确保输入格式正确
theta_samples = amortizer.sample(input_dict, n_samples=500, to_numpy=True)  


# 后验均值
posterior_means = theta_samples.mean(axis=1)  # => [test_size, 47]

# 与真参数对比
theta_true_np = theta_true.numpy()  # => [test_size, 47]
mse = np.mean((posterior_means - theta_true_np)**2)
print("Posterior mean MSE =", mse)


Posterior mean MSE = 1.5065843


In [31]:
import numpy as np

posterior_means = theta_samples.mean(axis=1)    # [test_size, 47]
posterior_vars  = theta_samples.var(axis=1)     # [test_size, 47]
mse_per_dim = np.mean((posterior_means - theta_true)**2, axis=0)
print("MSE per dimension (47 dims):", mse_per_dim)


MSE per dimension (47 dims): [1.6366448  2.63546    1.0594751  0.76740676 2.1097806  1.956761
 1.2554277  2.8740022  1.0849876  1.1856735  1.1016594  2.0459263
 0.7443749  0.79282016 2.371237   0.7638977  1.7302583  2.0397904
 1.1147163  1.11868    1.0639788  1.311651   0.9358189  1.0053694
 1.8535951  1.2763115  1.7377216  1.1116188  1.4587002  1.4296274
 1.4556426  1.0309477  1.3826393  1.1353819  1.6988935  1.7592765
 1.7632159  1.2828435  1.6004134  1.4990789  1.985598   1.4621035
 1.588435   2.091395   1.4444824  1.5464802  2.5092623 ]


In [32]:
bias = np.mean(posterior_means - theta_true, axis=0)  # shape [47]
print("Mean bias across test set, per dimension:", bias)


Mean bias across test set, per dimension: [-0.04909414 -0.01510814  0.02551518  0.04869845 -0.03131106  0.02827244
 -0.00889206  0.0361018  -0.02500717 -0.04163445 -0.02441601 -0.00163136
  0.05812452 -0.01475854  0.01750979  0.00263184  0.02113112  0.08130535
  0.03107602 -0.05161801  0.02051635 -0.0118619   0.01025875 -0.0351148
 -0.01610831  0.00487701  0.0106229  -0.04800817 -0.0523546   0.02818106
 -0.03560602  0.03050803  0.02203771  0.0173383  -0.00810698 -0.02965671
 -0.03091044  0.02235381  0.00767428  0.06824006 -0.0335241   0.01413976
 -0.03640479  0.04791142 -0.04082818  0.03967031 -0.04870167]
